## SMART TRAFFIC LIGHT CONTROL SYSTEM

In [22]:
import numpy as np
import skfuzzy as fuzz
from skfuzzy import control as ctrl
import matplotlib
matplotlib.use('Agg')
import matplotlib.pyplot as plt
import pandas as pd
from sklearn.neural_network import MLPRegressor  # For neuro-fuzzy training
import time

In [23]:
def generate_realistic_traffic_data(n_samples=10000):
    np.random.seed(42)
    data = {
        'vehicle_density': [],
        'waiting_time': [],
        'flow_rate': [],
        'queue_length': [],
        'emergency_vehicle': [],
        'opposing_density': [],
        'green_duration': []
    }
    for _ in range(n_samples):
        period = np.random.choice(['rush', 'off-peak', 'night'], p=[0.3, 0.5, 0.2])
        if period == 'rush':
            density = np.random.randint(70, 100)
            wait = np.random.randint(40, 60)
            flow = np.random.randint(40, 60)
            queue = np.random.randint(20, 30)
            emergency = np.random.choice([0, 1], p=[0.95, 0.05])  # 5% chance
            opp_density = np.random.randint(50, 100)
            green = np.random.randint(45, 60)
        elif period == 'off-peak':
            density = np.random.randint(20, 70)
            wait = np.random.randint(10, 40)
            flow = np.random.randint(20, 40)
            queue = np.random.randint(5, 20)
            emergency = np.random.choice([0, 1], p=[0.98, 0.02])  # 2% chance
            opp_density = np.random.randint(20, 70)
            green = np.random.randint(20, 45)
        else:  # nighttime
            density = np.random.randint(0, 20)
            wait = np.random.randint(0, 15)
            flow = np.random.randint(0, 20)
            queue = np.random.randint(0, 10)
            emergency = np.random.choice([0, 1], p=[0.99, 0.01])  # 1% chance
            opp_density = np.random.randint(0, 20)
            green = np.random.randint(10, 20)
        data['vehicle_density'].append(density)
        data['waiting_time'].append(wait)
        data['flow_rate'].append(flow)
        data['queue_length'].append(queue)
        data['emergency_vehicle'].append(emergency)
        data['opposing_density'].append(opp_density)
        data['green_duration'].append(green)
    return pd.DataFrame(data)

In [24]:
# Load or generate dataset
data = generate_realistic_traffic_data(n_samples=10000)
data.to_csv('traffic_data_10000.csv', index=False)  # Save for reference

# Step 2: Load Dataset from CSV
try:
    data = pd.read_csv('traffic_data_10000.csv')
    print(f"Loaded {len(data)} samples from 'traffic_data_10000.csv'")
except FileNotFoundError:
    print("Error: 'traffic_data_10000.csv' not found. Please ensure the file exists.")
    exit(1)

Loaded 10000 samples from 'traffic_data_10000.csv'


### Step 1: Define Fuzzy Variables and Membership Functions

In [25]:
# Input: Vehicle Density (0 to 100 vehicles)
vehicle_density = ctrl.Antecedent(np.arange(0, 101, 1), 'vehicle_density')
vehicle_density['low'] = fuzz.trimf(vehicle_density.universe, [0, 0, 50])
vehicle_density['medium'] = fuzz.trimf(vehicle_density.universe, [25, 50, 75])
vehicle_density['high'] = fuzz.trimf(vehicle_density.universe, [50, 100, 100])

In [26]:
# Input: Waiting Time (0 to 60 seconds)
waiting_time = ctrl.Antecedent(np.arange(0, 61, 1), 'waiting_time')
waiting_time['short'] = fuzz.trimf(waiting_time.universe, [0, 0, 30])
waiting_time['medium'] = fuzz.trimf(waiting_time.universe, [15, 30, 45])
waiting_time['long'] = fuzz.trimf(waiting_time.universe, [30, 60, 60])

In [27]:
flow_rate = ctrl.Antecedent(np.arange(0, 61, 1), 'flow_rate')
flow_rate['low'] = fuzz.trimf(flow_rate.universe, [0, 0, 25])  # Widened overlap
flow_rate['medium'] = fuzz.trimf(flow_rate.universe, [15, 35, 55])
flow_rate['high'] = fuzz.trimf(flow_rate.universe, [45, 60, 60])

In [28]:
queue_length = ctrl.Antecedent(np.arange(0, 31, 1), 'queue_length')
queue_length['short'] = fuzz.trimf(queue_length.universe, [0, 0, 12])  # Widened overlap
queue_length['medium'] = fuzz.trimf(queue_length.universe, [8, 18, 28])
queue_length['long'] = fuzz.trimf(queue_length.universe, [22, 30, 30])

In [29]:
emergency_vehicle = ctrl.Antecedent(np.arange(0, 2, 1), 'emergency_vehicle')
emergency_vehicle['absent'] = fuzz.trimf(emergency_vehicle.universe, [0, 0, 0])
emergency_vehicle['present'] = fuzz.trimf(emergency_vehicle.universe, [1, 1, 1])

In [30]:
opposing_density = ctrl.Antecedent(np.arange(0, 101, 1), 'opposing_density')
opposing_density['low'] = fuzz.trimf(opposing_density.universe, [0, 0, 50])
opposing_density['medium'] = fuzz.trimf(opposing_density.universe, [25, 50, 75])
opposing_density['high'] = fuzz.trimf(opposing_density.universe, [50, 100, 100])

In [31]:
# Output: Green Light Duration (10 to 60 seconds)
green_duration = ctrl.Consequent(np.arange(10, 61, 1), 'green_duration')
green_duration['short'] = fuzz.trimf(green_duration.universe, [10, 10, 30])
green_duration['medium'] = fuzz.trimf(green_duration.universe, [20, 35, 50])
green_duration['long'] = fuzz.trimf(green_duration.universe, [40, 60, 60])

### Step 2: Define Fuzzy Rules

In [32]:
rule1 = ctrl.Rule(vehicle_density['high'] & waiting_time['long'] & flow_rate['high'] & queue_length['long'] & emergency_vehicle['absent'] & opposing_density['low'], green_duration['long'])
rule2 = ctrl.Rule(vehicle_density['medium'] & waiting_time['medium'] & flow_rate['medium'] & queue_length['medium'] & emergency_vehicle['absent'] & opposing_density['medium'], green_duration['medium'])
rule3 = ctrl.Rule(vehicle_density['low'] & waiting_time['short'] & flow_rate['low'] & queue_length['short'] & emergency_vehicle['absent'] & opposing_density['high'], green_duration['short'])
rule4 = ctrl.Rule(emergency_vehicle['present'], green_duration['long'])  # Priority for emergency vehicles
rule5 = ctrl.Rule(flow_rate['high'] & queue_length['long'] & opposing_density['low'], green_duration['long'])
rule6 = ctrl.Rule(opposing_density['high'] & vehicle_density['low'], green_duration['short'])
rule7 = ctrl.Rule(vehicle_density['high'] & queue_length['short'] & flow_rate['medium'], green_duration['medium'])  # Mixed case
rule8 = ctrl.Rule(waiting_time['long'] & flow_rate['low'] & queue_length['medium'], green_duration['medium'])  # Mixed case
rule9 = ctrl.Rule(vehicle_density['low'] & opposing_density['low'] & emergency_vehicle['absent'], green_duration['short'])  # Low traffic case

### Step 3: Create Fuzzy Control System

In [33]:
traffic_ctrl = ctrl.ControlSystem([rule1, rule2, rule3, rule4, rule5, rule6])
traffic_sim = ctrl.ControlSystemSimulation(traffic_ctrl)


sample_sizes = [100, 500, 1000, 2500, 5000, 7500 ,10000]
performance_metrics = {
    'sample_size': [],
    'avg_wait_time': [],
    'training_time': []
}

In [34]:
for n_samples in sample_sizes:
    print(f"\nEvaluating performance with {n_samples} samples...")
    subset_data = data.sample(n=n_samples, random_state=42)
    train_data = subset_data.sample(frac=0.8, random_state=42)
    test_data = subset_data.drop(train_data.index)
    
    start_time = time.time()
    X_train = train_data[['vehicle_density', 'waiting_time', 'flow_rate', 'queue_length', 'emergency_vehicle', 'opposing_density']].values
    y_train = train_data['green_duration'].values
    mlp = MLPRegressor(hidden_layer_sizes=(10, 10), max_iter=1000, random_state=42)
    mlp.fit(X_train, y_train)
    training_time = time.time() - start_time
    
    wait_times = []
    predicted_green_durations = []
    for i in test_data.index:
        # Input validation
        inputs = {
            'vehicle_density': np.clip(test_data.loc[i, 'vehicle_density'], 0, 100),
            'waiting_time': np.clip(test_data.loc[i, 'waiting_time'], 0, 60),
            'flow_rate': np.clip(test_data.loc[i, 'flow_rate'], 0, 60),
            'queue_length': np.clip(test_data.loc[i, 'queue_length'], 0, 30),
            'emergency_vehicle': np.clip(test_data.loc[i, 'emergency_vehicle'], 0, 1),
            'opposing_density': np.clip(test_data.loc[i, 'opposing_density'], 0, 100)
        }
        # Check for NaN or invalid values
        if any(np.isnan(val) for val in inputs.values()):
            print(f"Warning: Invalid input for sample {i}: {inputs}")
            predicted_green_durations.append(10)
            wait_times.append(test_data.loc[i, 'waiting_time'])
            continue
        traffic_sim.input['vehicle_density'] = test_data.loc[i, 'vehicle_density']
        traffic_sim.input['waiting_time'] = test_data.loc[i, 'waiting_time']
        traffic_sim.input['flow_rate'] = test_data.loc[i, 'flow_rate']
        traffic_sim.input['queue_length'] = test_data.loc[i, 'queue_length']
        traffic_sim.input['emergency_vehicle'] = test_data.loc[i, 'emergency_vehicle']
        traffic_sim.input['opposing_density'] = test_data.loc[i, 'opposing_density']
        try:
            traffic_sim.compute()
            fuzzy_green = traffic_sim.output['green_duration']
            nn_green = mlp.predict([[test_data.loc[i, 'vehicle_density'], test_data.loc[i, 'waiting_time'], 
                                    test_data.loc[i, 'flow_rate'], test_data.loc[i, 'queue_length'], 
                                    test_data.loc[i, 'emergency_vehicle'], test_data.loc[i, 'opposing_density']]])[0]
            green_time = 0.7 * fuzzy_green + 0.3 * nn_green
            green_time = np.clip(green_time, 10, 60)
            predicted_green_durations.append(green_time)
            wait_time = max(0, test_data.loc[i, 'waiting_time'] - green_time)
            wait_times.append(wait_time)
        except Exception as e:
            print(f"Error during simulation for sample {i}: {e}")
            print(f"Inputs: {inputs}")
            predicted_green_durations.append(10)
            wait_times.append(test_data.loc[i, 'waiting_time'])
    
    avg_wait_time = np.mean(wait_times)
    performance_metrics['sample_size'].append(n_samples)
    performance_metrics['avg_wait_time'].append(avg_wait_time)
    performance_metrics['training_time'].append(training_time)
    print(f"Sample Size: {n_samples}, Avg Wait Time: {avg_wait_time:.2f} seconds, Training Time: {training_time:.2f} seconds")
    # Generate and save scatter plot for this subset
    plt.figure(figsize=(10, 6))
    plt.scatter(test_data['vehicle_density'], test_data['waiting_time'], c=predicted_green_durations, cmap='viridis')
    plt.colorbar(label='Green Light Duration (seconds)')
    plt.xlabel('Vehicle Density')
    plt.ylabel('Waiting Time (seconds)')
    plt.title(f'Traffic Light Control: Green Light Duration ({n_samples} Samples, Test Set)')
    plt.savefig(f'traffic_plot_{n_samples}.png')
    plt.close()
    print(f"Traffic plot for {n_samples} samples saved to 'traffic_plot_{n_samples}.png'")


Evaluating performance with 100 samples...


c:\Python312\Lib\site-packages\sklearn\neural_network\_multilayer_perceptron.py:691: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (1000) reached and the optimization hasn't converged yet.
  warnings.warn(


Error during simulation for sample 4684: 'green_duration'
Inputs: {'vehicle_density': np.int64(15), 'waiting_time': np.int64(12), 'flow_rate': np.int64(7), 'queue_length': np.int64(1), 'emergency_vehicle': np.int64(0), 'opposing_density': np.int64(14)}
Error during simulation for sample 3999: 'green_duration'
Inputs: {'vehicle_density': np.int64(92), 'waiting_time': np.int64(51), 'flow_rate': np.int64(48), 'queue_length': np.int64(22), 'emergency_vehicle': np.int64(0), 'opposing_density': np.int64(75)}
Error during simulation for sample 9896: 'green_duration'
Inputs: {'vehicle_density': np.int64(15), 'waiting_time': np.int64(5), 'flow_rate': np.int64(3), 'queue_length': np.int64(1), 'emergency_vehicle': np.int64(0), 'opposing_density': np.int64(18)}
Error during simulation for sample 2249: 'green_duration'
Inputs: {'vehicle_density': np.int64(21), 'waiting_time': np.int64(21), 'flow_rate': np.int64(35), 'queue_length': np.int64(7), 'emergency_vehicle': np.int64(0), 'opposing_density': 

c:\Python312\Lib\site-packages\sklearn\neural_network\_multilayer_perceptron.py:691: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (1000) reached and the optimization hasn't converged yet.
  warnings.warn(


Error during simulation for sample 8328: 'green_duration'
Inputs: {'vehicle_density': np.int64(3), 'waiting_time': np.int64(0), 'flow_rate': np.int64(9), 'queue_length': np.int64(4), 'emergency_vehicle': np.int64(0), 'opposing_density': np.int64(4)}
Error during simulation for sample 1513: 'green_duration'
Inputs: {'vehicle_density': np.int64(93), 'waiting_time': np.int64(55), 'flow_rate': np.int64(53), 'queue_length': np.int64(22), 'emergency_vehicle': np.int64(0), 'opposing_density': np.int64(95)}
Error during simulation for sample 2304: 'green_duration'
Inputs: {'vehicle_density': np.int64(71), 'waiting_time': np.int64(46), 'flow_rate': np.int64(58), 'queue_length': np.int64(26), 'emergency_vehicle': np.int64(0), 'opposing_density': np.int64(63)}
Error during simulation for sample 3723: 'green_duration'
Inputs: {'vehicle_density': np.int64(93), 'waiting_time': np.int64(47), 'flow_rate': np.int64(41), 'queue_length': np.int64(28), 'emergency_vehicle': np.int64(0), 'opposing_density':

In [35]:
performance_df = pd.DataFrame(performance_metrics)
performance_df.to_csv('performance_metrics.csv', index=False)
print("\nPerformance metrics saved to 'performance_metrics.csv'")


Performance metrics saved to 'performance_metrics.csv'


In [36]:
if not performance_metrics['sample_size']:
    print("Error: No performance metrics to plot. Check data generation.")
else:
    print("Performance Metrics:", performance_metrics)
    plt.figure(figsize=(12, 6))
    plt.subplot(1, 2, 1)
    plt.plot(performance_metrics['sample_size'], performance_metrics['avg_wait_time'], marker='o')
    plt.xlabel('Sample Size')
    plt.ylabel('Average Waiting Time (seconds)')
    plt.title('Average Waiting Time vs. Sample Size')
    plt.grid(True)
    plt.subplot(1, 2, 2)
    plt.plot(performance_metrics['sample_size'], performance_metrics['training_time'], marker='o', color='orange')
    plt.xlabel('Sample Size')
    plt.ylabel('Training Time (seconds)')
    plt.title('Training Time vs. Sample Size')
    plt.grid(True)
    plt.tight_layout()
    plt.savefig('performance_plot.png')
    plt.close()
    print("Performance plot saved to 'performance_plot.png'")

Performance Metrics: {'sample_size': [100, 500, 1000, 2500, 5000, 7500, 10000], 'avg_wait_time': [np.float64(21.64561017180476), np.float64(20.042996575278213), np.float64(17.266292643728484), np.float64(17.446300937559585), np.float64(17.935856233526028), np.float64(17.955343862151203), np.float64(20.145889083519783)], 'training_time': [0.4447052478790283, 1.2392807006835938, 0.8530135154724121, 1.7277297973632812, 1.2349631786346436, 0.9994616508483887, 1.6977062225341797]}
Performance plot saved to 'performance_plot.png'


### Step 4: Train Neuro-Fuzzy System (Simplified)

In [37]:
# Split data into training (80%) and testing (20%)
train_data = data.sample(frac=0.8, random_state=42)
test_data = data.drop(train_data.index)

In [38]:
X_train = train_data[['vehicle_density', 'waiting_time', 'flow_rate', 'queue_length', 'emergency_vehicle', 'opposing_density']].values
y_train = train_data['green_duration'].values
mlp = MLPRegressor(hidden_layer_sizes=(10, 10), max_iter=1000, random_state=42)
mlp.fit(X_train, y_train)

MLPRegressor(hidden_layer_sizes=(10, 10), max_iter=1000, random_state=42)

### Step 5: Test the System

In [39]:
wait_times = []
predicted_green_durations = []
test_results = {
    'vehicle_density': [],
    'waiting_time': [],
    'flow_rate': [],
    'queue_length': [],
    'emergency_vehicle': [],
    'opposing_density': [],
    'predicted_green_duration': [],
    'actual_wait_time': []
}
for i in test_data.index:
    # Input validation
    inputs = {
        'vehicle_density': np.clip(test_data.loc[i, 'vehicle_density'], 0, 100),
        'waiting_time': np.clip(test_data.loc[i, 'waiting_time'], 0, 60),
        'flow_rate': np.clip(test_data.loc[i, 'flow_rate'], 0, 60),
        'queue_length': np.clip(test_data.loc[i, 'queue_length'], 0, 30),
        'emergency_vehicle': np.clip(test_data.loc[i, 'emergency_vehicle'], 0, 1),
        'opposing_density': np.clip(test_data.loc[i, 'opposing_density'], 0, 100)
    }
    if any(np.isnan(val) for val in inputs.values()):
        print(f"Warning: Invalid input for sample {i}: {inputs}")
        predicted_green_durations.append(10)
        wait_times.append(test_data.loc[i, 'waiting_time'])
        test_results['vehicle_density'].append(test_data.loc[i, 'vehicle_density'])
        test_results['waiting_time'].append(test_data.loc[i, 'waiting_time'])
        test_results['flow_rate'].append(test_data.loc[i, 'flow_rate'])
        test_results['queue_length'].append(test_data.loc[i, 'queue_length'])
        test_results['emergency_vehicle'].append(test_data.loc[i, 'emergency_vehicle'])
        test_results['opposing_density'].append(test_data.loc[i, 'opposing_density'])
        test_results['predicted_green_duration'].append(10)
        test_results['actual_wait_time'].append(test_data.loc[i, 'waiting_time'])
        continue
    
    traffic_sim.input['vehicle_density'] = test_data.loc[i, 'vehicle_density']
    traffic_sim.input['waiting_time'] = test_data.loc[i, 'waiting_time']
    traffic_sim.input['flow_rate'] = test_data.loc[i, 'flow_rate']
    traffic_sim.input['queue_length'] = test_data.loc[i, 'queue_length']
    traffic_sim.input['emergency_vehicle'] = test_data.loc[i, 'emergency_vehicle']
    traffic_sim.input['opposing_density'] = test_data.loc[i, 'opposing_density']
    try:
        traffic_sim.compute()
        fuzzy_green = traffic_sim.output['green_duration']
        nn_green = mlp.predict([[test_data.loc[i, 'vehicle_density'], test_data.loc[i, 'waiting_time'], 
                                test_data.loc[i, 'flow_rate'], test_data.loc[i, 'queue_length'], 
                                test_data.loc[i, 'emergency_vehicle'], test_data.loc[i, 'opposing_density']]])[0]
        green_time = 0.7 * fuzzy_green + 0.3 * nn_green
        green_time = np.clip(green_time, 10, 60)
        wait_time = max(0, test_data.loc[i, 'waiting_time'] - green_time)
        
        predicted_green_durations.append(green_time)
        wait_times.append(wait_time)
        test_results['vehicle_density'].append(test_data.loc[i, 'vehicle_density'])
        test_results['waiting_time'].append(test_data.loc[i, 'waiting_time'])
        test_results['flow_rate'].append(test_data.loc[i, 'flow_rate'])
        test_results['queue_length'].append(test_data.loc[i, 'queue_length'])
        test_results['emergency_vehicle'].append(test_data.loc[i, 'emergency_vehicle'])
        test_results['opposing_density'].append(test_data.loc[i, 'opposing_density'])
        test_results['predicted_green_duration'].append(green_time)
        test_results['actual_wait_time'].append(wait_time)
    except Exception as e:
        print(f"Error during simulation for sample {i}: {e}")
        print(f"Inputs: {inputs}")
        predicted_green_durations.append(10)
        wait_times.append(test_data.loc[i, 'waiting_time'])
        test_results['vehicle_density'].append(test_data.loc[i, 'vehicle_density'])
        test_results['waiting_time'].append(test_data.loc[i, 'waiting_time'])
        test_results['flow_rate'].append(test_data.loc[i, 'flow_rate'])
        test_results['queue_length'].append(test_data.loc[i, 'queue_length'])
        test_results['emergency_vehicle'].append(test_data.loc[i, 'emergency_vehicle'])
        test_results['opposing_density'].append(test_data.loc[i, 'opposing_density'])
        test_results['predicted_green_duration'].append(10)
        test_results['actual_wait_time'].append(test_data.loc[i, 'waiting_time'])

Error during simulation for sample 1: 'green_duration'
Inputs: {'vehicle_density': np.int64(80), 'waiting_time': np.int64(43), 'flow_rate': np.int64(47), 'queue_length': np.int64(27), 'emergency_vehicle': np.int64(0), 'opposing_density': np.int64(51)}
Error during simulation for sample 4: 'green_duration'
Inputs: {'vehicle_density': np.int64(40), 'waiting_time': np.int64(18), 'flow_rate': np.int64(26), 'queue_length': np.int64(6), 'emergency_vehicle': np.int64(0), 'opposing_density': np.int64(33)}
Error during simulation for sample 5: 'green_duration'
Inputs: {'vehicle_density': np.int64(45), 'waiting_time': np.int64(30), 'flow_rate': np.int64(21), 'queue_length': np.int64(8), 'emergency_vehicle': np.int64(0), 'opposing_density': np.int64(26)}
Error during simulation for sample 38: 'green_duration'
Inputs: {'vehicle_density': np.int64(8), 'waiting_time': np.int64(4), 'flow_rate': np.int64(16), 'queue_length': np.int64(4), 'emergency_vehicle': np.int64(0), 'opposing_density': np.int64(0

In [40]:
test_results_df = pd.DataFrame(test_results)
test_results_df.to_csv('test_results_10000.csv', index=False)
print("Full test set results saved to 'test_results_5000.csv'")

Full test set results saved to 'test_results_5000.csv'


### Step 6: Evaluate Performance

In [41]:
avg_wait_time = np.mean(wait_times)
with open('traffic_results_10000.txt', 'w') as f:
    f.write(f"Average Waiting Time (Test Set): {avg_wait_time:.2f} seconds\n")
    f.write(f"Number of Training Samples: {len(train_data)}\n")
    f.write(f"Number of Test Samples: {len(test_data)}\n")
    f.write("Sample Test Data (Vehicle Density, Waiting Time, Flow Rate, Queue Length, Emergency Vehicle, Opposing Density, Predicted Green Duration):\n")
    for i, idx in enumerate(test_data.index[:10]):
        f.write(f"{test_data.loc[idx, 'vehicle_density']}, {test_data.loc[idx, 'waiting_time']}, "
                f"{test_data.loc[idx, 'flow_rate']}, {test_data.loc[idx, 'queue_length']}, "
                f"{test_data.loc[idx, 'emergency_vehicle']}, {test_data.loc[idx, 'opposing_density']}, "
                f"{predicted_green_durations[i]:.2f}\n")
print("Detailed results saved to 'traffic_results_10000.txt'")



Detailed results saved to 'traffic_results_10000.txt'


In [42]:
plt.figure(figsize=(10, 6))
plt.scatter(test_data['vehicle_density'], test_data['waiting_time'], c=predicted_green_durations, cmap='viridis')
plt.colorbar(label='Green Light Duration (seconds)')
plt.xlabel('Vehicle Density')
plt.ylabel('Waiting Time (seconds)')
plt.title('Traffic Light Control: Green Light Duration (10000 Samples, Test Set)')
plt.savefig('traffic_plot_10000.png')
plt.close()
print("Traffic plot saved to 'traffic_plot_5000.png'")

Traffic plot saved to 'traffic_plot_5000.png'
